In [ ]:
!pip install pyspark

In [ ]:
!gdown --id 1n8pRtb_V_zc-Xk0ODm-LkDIXT6brkaaV

Downloading...
From: https://drive.google.com/uc?id=1n8pRtb_V_zc-Xk0ODm-LkDIXT6brkaaV
To: /content/newspapers.zip
100% 874k/874k [00:00<00:00, 16.3MB/s]


In [ ]:
# https://drive.google.com/file/d/1n8pRtb_V_zc-Xk0ODm-LkDIXT6brkaaV/view?usp=sharing

In [ ]:
!unzip /content/newspapers.zip

Archive:  /content/newspapers.zip
replace newspapers/1.136 hồ sơ ứng cử đại biểu Quốc hội.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: newspapers/1.136 hồ sơ ứng cử đại biểu Quốc hội.txt  
replace newspapers/1.500 tỷ làm đường vành đai hai năm vẫn ngổn ngang, Đà Nẵng thanh tra gấp.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace newspapers/1.500 tỷ làm đường vành đai hai năm vẫn ngổn ngang, Đà Nẵng thanh tra gấp.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: newspapers/1.500 tỷ làm đường vành đai hai năm vẫn ngổn ngang, Đà Nẵng thanh tra gấp.txt  
  inflating: newspapers/2 tàu cá Việt Nam với gần 40 ngư dân bị Indonesia bắt giữ.txt  
  inflating: newspapers/3 ôtô tông liên hoàn ở cửa ngõ Sài Gòn.txt  
  inflating: newspapers/4 ôtô tông liên hoàn khi dừng đèn đỏ.txt  
  inflating: newspapers/5 bài học kinh nghiệm trong điều hành của Chính phủ.txt  
  inflating: newspapers/5 ôtô tông liên hoàn trên quốc lộ 51.txt  
  inflating: new

In [ ]:
from pyspark import SparkConf, SparkContext
from nltk import word_tokenize
from nltk.util import ngrams
from random import shuffle
import pyspark
import nltk
import numpy as np
import re
import os
import time

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sc = SparkContext(master="local", appName="SimilarNewspapers")

In [ ]:
newspapers = os.listdir('newspapers/')
print(len(newspapers))

497


In [ ]:
data = sc.wholeTextFiles('newspapers')
data.collect()[0]

("file:/content/newspapers/9 dự án BOT xuống cấp không ai sửa, Bộ GTVT 'loay hoay' tìm giải pháp.txt",
 "https://vietnamnet.vn/vn/thoi-su/vn/thoi-su/an-toan-giao-thong/9-du-an-bot-xuong-cap-khong-ai-sua-bo-gtvt-loay-hoay-tim-giai-phap-720967.html\r\n9 dự án BOT xuống cấp không ai sửa, Bộ GTVT 'loay hoay' tìm giải pháp\r\nBộ GTVT yêu cầu Tổng cục Đường bộ đề xuất giải pháp tạm thời để xử lý tình huống vướng mắc trong quản lý, bảo trì đối với các dự án BOT đang tạm dừng thu phí.Bộ GTVT vừa yêu cầu Tổng cục Đường bộ Việt Nam có giải pháp tham mưu cho lãnh đạo Bộ để quản lý, bảo trì các tuyến đường đầu tư theo hình thức BOT trong thời gian dừng thu phí đến khi hoàn tất thủ tục xác lập quyền sở hữu toàn dân về tài sản...Tham mưu cho Bộ giải pháp tạm thời để xử lý tình huống một số vướng mắc trong quản lý, bảo trì đối với những tuyến đường đầu tư theo hình thức BOT trong thời gian tạm dừng thu phí, đảm bảo đúng theo quy định pháp luật.Trong đó, cần lưu ý những phương án có thể thực hiện ngay

In [ ]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.lower()
    text = re.sub('\W+', ' ', text)
    text = text.strip()
    
    return text
  
clean_text('This is a  a  test \n asdklf      \t asdf')

'this is a a test asdklf asdf'

In [ ]:
data = data.map(lambda abc: (abc[0], abc[1].split("\n")))
data.collect()[0]
data = data.map(lambda abc: (abc[0], clean_text(abc[1][1]+abc[1][2])))
data.collect()[0]

("file:/content/newspapers/9 dự án BOT xuống cấp không ai sửa, Bộ GTVT 'loay hoay' tìm giải pháp.txt",
 '9 dự án bot xuống cấp không ai sửa bộ gtvt loay hoay tìm giải pháp bộ gtvt yêu cầu tổng cục đường bộ đề xuất giải pháp tạm thời để xử lý tình huống vướng mắc trong quản lý bảo trì đối với các dự án bot đang tạm dừng thu phí bộ gtvt vừa yêu cầu tổng cục đường bộ việt nam có giải pháp tham mưu cho lãnh đạo bộ để quản lý bảo trì các tuyến đường đầu tư theo hình thức bot trong thời gian dừng thu phí đến khi hoàn tất thủ tục xác lập quyền sở hữu toàn dân về tài sản tham mưu cho bộ giải pháp tạm thời để xử lý tình huống một số vướng mắc trong quản lý bảo trì đối với những tuyến đường đầu tư theo hình thức bot trong thời gian tạm dừng thu phí đảm bảo đúng theo quy định pháp luật trong đó cần lưu ý những phương án có thể thực hiện ngay công tác bảo quản đảm bảo atgt trong giai đoạn này đường tránh bot thanh hoá bị hằn lún nhưng mãi không sửa đi lại rất nguy hiểmtổng cục đường bộ cho biết hi

**Step 1: Shingling**

In [ ]:
def calculate_join_grams(text, n):
    words = word_tokenize(text)
    n_gram = ngrams(words, n)
    join_grams = [' '.join(list(gram)) for gram in n_gram]
    
    return join_grams
  
calculate_join_grams('This is a test', n=3)

['This is a', 'is a test']

In [ ]:
start_time = time.time()

shingles = data.flatMap(lambda abc: calculate_join_grams(abc[1], 3))
shingles = shingles.distinct()
shingles = shingles.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(shingles))

--- 3.9961960315704346 seconds ---
178922


In [ ]:
def create_newspaper_vector(cleaned_title_and_content):   
    exist_booleans = list()
    for shingle in shingles:
        if cleaned_title_and_content.find(shingle) != -1:
            exist_booleans.append(1)
        else:
            exist_booleans.append(0)
            
    return exist_booleans

In [ ]:
start_time = time.time()

newspaper_vectors = data.map(lambda abc: create_newspaper_vector(abc[1]))
newspaper_vectors = newspaper_vectors.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(newspaper_vectors))

--- 1.1261012554168701 seconds ---
497


**Step 2: Minhashing**

In [ ]:
def create_permutation(number_shingles):
    random_permutation_numbers = [i for i in range(1, number_shingles + 1)]
    shuffle(random_permutation_numbers)
    return random_permutation_numbers


In [24]:
start_time = time.time()

number_permutation = 100
number_shingles = len(shingles)

rdd_create_permutation = sc.parallelize([x for x in range(0, number_permutation)])
permutations = rdd_create_permutation.map(lambda abc: create_permutation(number_shingles))

permutations = permutations.collect()


print(f'--- {time.time() - start_time} seconds ---')
print(len(permutations))

--- 1.1220216751098633 seconds ---
100


In [25]:
def create_signature_vector(newspaper_vector):
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
            
    return signature_vector

In [26]:
start_time = time.time()

rdd_newspaper_vectors = sc.parallelize(newspaper_vectors)

signature_vectors = rdd_newspaper_vectors.map(lambda x: create_signature_vector(x))
signature_vectors = signature_vectors.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(signature_vectors))

[[3, 13, 21, 10, 9, 36, 12, 8, 20, 24, 37, 11, 4, 20, 79, 4, 4, 10, 27, 5, 13, 9, 36, 4, 17, 2, 6, 50, 49, 20, 1, 31, 8, 18, 45, 21, 1, 13, 14, 33, 3, 2, 6, 1, 18, 28, 96, 43, 7, 44, 3, 21, 5, 9, 40, 55, 8, 17, 17, 8, 2, 7, 4, 29, 97, 11, 4, 56, 7, 22, 5, 13, 25, 14, 21, 4, 38, 54, 3, 74, 13, 16, 20, 11, 3, 28, 18, 9, 23, 14, 10, 26, 12, 59, 13, 39, 92, 30, 133, 6], [17, 17, 22, 34, 1, 3, 18, 4, 34, 28, 27, 6, 13, 44, 23, 31, 28, 22, 21, 28, 11, 2, 18, 14, 10, 20, 3, 3, 11, 1, 18, 6, 50, 1, 5, 2, 15, 16, 41, 6, 2, 15, 28, 18, 11, 5, 12, 2, 38, 24, 17, 2, 10, 16, 6, 7, 32, 15, 40, 3, 22, 39, 34, 27, 22, 3, 36, 21, 55, 21, 46, 30, 5, 4, 20, 21, 51, 1, 5, 23, 23, 31, 11, 23, 15, 1, 25, 15, 1, 22, 11, 24, 5, 16, 37, 22, 5, 12, 7, 10], [114, 8, 27, 6, 16, 1, 2, 2, 4, 41, 18, 64, 75, 13, 3, 7, 15, 5, 14, 27, 4, 3, 17, 72, 51, 12, 15, 34, 13, 6, 27, 26, 33, 4, 38, 12, 39, 6, 18, 2, 44, 4, 43, 22, 15, 41, 8, 25, 42, 5, 26, 25, 80, 32, 11, 24, 59, 26, 6, 59, 8, 71, 51, 44, 10, 13, 5, 20, 23, 16

**Step 3: Locality Sensitive Hashing**

In [27]:
number_of_band = 20
number_of_row = int(len(permutations) / number_of_band)
print(number_of_band, number_of_row)

20 5


In [47]:
start_time = time.time()

buckets = dict()

for index_newspaper, signature_vector in enumerate(signature_vectors):
    previous_row = 0
    for _ in range(number_of_band):
        key_chars = list()
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key not in buckets:
            buckets[key] = [index_newspaper]
        else:
            buckets[key].append(index_newspaper)

print(f'--- {time.time() - start_time} seconds ---')
print(buckets)
print(len(buckets))

--- 0.0031304359436035156 seconds ---
{'3_13_21_10_9': [0], '36_12_8_20_24': [0], '37_11_4_20_79': [0], '4_4_10_27_5': [0], '13_9_36_4_17': [0], '2_6_50_49_20': [0], '1_31_8_18_45': [0], '21_1_13_14_33': [0], '3_2_6_1_18': [0], '28_96_43_7_44': [0], '3_21_5_9_40': [0], '55_8_17_17_8': [0], '2_7_4_29_97': [0], '11_4_56_7_22': [0], '5_13_25_14_21': [0], '4_38_54_3_74': [0], '13_16_20_11_3': [0], '28_18_9_23_14': [0], '10_26_12_59_13': [0], '39_92_30_133_6': [0], '17_17_22_34_1': [1], '3_18_4_34_28': [1], '27_6_13_44_23': [1], '31_28_22_21_28': [1], '11_2_18_14_10': [1], '20_3_3_11_1': [1], '18_6_50_1_5': [1], '2_15_16_41_6': [1], '2_15_28_18_11': [1], '5_12_2_38_24': [1], '17_2_10_16_6': [1], '7_32_15_40_3': [1], '22_39_34_27_22': [1], '3_36_21_55_21': [1], '46_30_5_4_20': [1], '21_51_1_5_23': [1], '23_31_11_23_15': [1], '1_25_15_1_22': [1], '11_24_5_16_37': [1], '22_5_12_7_10': [1], '114_8_27_6_16': [2], '1_2_2_4_41': [2], '18_64_75_13_3': [2], '7_15_5_14_27': [2], '4_3_17_72_51': [2], 

In [31]:
def create_bucket(signature_vector):
    buckets = dict()
    previous_row = 0
    for _ in range(number_of_band):
        key_chars = list()
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key not in buckets:
            buckets[key] = [index_newspaper]
        else:
            buckets[key].append(index_newspaper)
      
    return buckets


In [46]:
start_time = time.time()

rdd_signature_vectors = sc.parallelize(signature_vectors)

dict_buckets = rdd_signature_vectors.map(lambda x: create_bucket(x))
dict_buckets = dict_buckets.collect()
print(dict_buckets)

buckets = dict()
for bucket in dict_buckets:
  buckets.update(bucket)

print(f'--- {time.time() - start_time} seconds ---')
print(len(buckets))

[{'3_13_21_10_9': [5], '36_12_8_20_24': [5], '37_11_4_20_79': [5], '4_4_10_27_5': [5], '13_9_36_4_17': [5], '2_6_50_49_20': [5], '1_31_8_18_45': [5], '21_1_13_14_33': [5], '3_2_6_1_18': [5], '28_96_43_7_44': [5], '3_21_5_9_40': [5], '55_8_17_17_8': [5], '2_7_4_29_97': [5], '11_4_56_7_22': [5], '5_13_25_14_21': [5], '4_38_54_3_74': [5], '13_16_20_11_3': [5], '28_18_9_23_14': [5], '10_26_12_59_13': [5], '39_92_30_133_6': [5]}, {'17_17_22_34_1': [5], '3_18_4_34_28': [5], '27_6_13_44_23': [5], '31_28_22_21_28': [5], '11_2_18_14_10': [5], '20_3_3_11_1': [5], '18_6_50_1_5': [5], '2_15_16_41_6': [5], '2_15_28_18_11': [5], '5_12_2_38_24': [5], '17_2_10_16_6': [5], '7_32_15_40_3': [5], '22_39_34_27_22': [5], '3_36_21_55_21': [5], '46_30_5_4_20': [5], '21_51_1_5_23': [5], '23_31_11_23_15': [5], '1_25_15_1_22': [5], '11_24_5_16_37': [5], '22_5_12_7_10': [5]}, {'114_8_27_6_16': [5], '1_2_2_4_41': [5], '18_64_75_13_3': [5], '7_15_5_14_27': [5], '4_3_17_72_51': [5], '12_15_34_13_6': [5], '27_26_33_4

In [48]:
for _, bucket in buckets.items():
    if len(bucket) > 1:
        print(bucket)
        for index_newspaper in bucket:
            print(newspapers[index_newspaper])
        print()

**Đưa vào một bản tin, liệt kê ra những bản tin tương tự**

In [49]:
def get_similar_newspapers(title_and_content, threshold=0.8):
    cleaned_title_and_content = clean_text(title_and_content)

    exist_booleans = list()
    for shingle in shingles:
        if shingle in cleaned_title_and_content:
            exist_booleans.append(True)
        else:
            exist_booleans.append(False)
    
    newspaper_vector = np.array(exist_booleans, dtype=bool)
    
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
    
    same_bucket_index_newspapers = set()
    
    key_chars = list()
    previous_row = 0
    for _ in range(number_of_band):
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key in buckets:
            same_bucket_index_newspapers.update(buckets[key])
        
    similar_index_newspapers = list()
    
    for same_bucket_index_newspaper in same_bucket_index_newspapers:
        count = 0
        for i, v in enumerate(newspaper_vector):
            if i > len(newspaper_vectors[same_bucket_index_newspaper]) - 1:
                break
                
            if newspaper_vectors[same_bucket_index_newspaper][i] == v:
                count += 1
        
        jaccard_similarity = count / len(newspaper_vectors[same_bucket_index_newspaper])
        # print(jaccard_similarity)

        if jaccard_similarity >= threshold:
            similar_index_newspapers.append(same_bucket_index_newspaper)
            
    return similar_index_newspapers


text = '''
áng 22/3, phiên họp thứ 4 Hội đồng bầu cử quốc gia diễn ra tại Nhà Quốc hội. Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia chủ trì phiên họp.Đến nay, Hội đồng bầu cử Quốc gia đã nhận được 1.136 hồ sơ người ứng cử ĐBQH khóa XV.Tổng Thư ký, Chủ nhiệm Văn phòng Quốc hội Nguyễn Hạnh Phúc, Chánh Văn phòng Hội đồng bầu cử quốc gia cho biết, thống kê sơ bộ, cho đến thời điểm này, Hội đồng bầu cử Quốc gia đã nhận được 1.136 hồ sơ người ứng cử ĐBQH khóa XV và 7.495 hồ sơ của người ứng cử đại biểu HĐND cấp tỉnh.Tính đến hết ngày 19/3, Ủy ban Mặt trận Tổ quốc các cấp đã hoàn thành việc tổ chức Hội nghị Hiệp thương lần thứ 2 để lập danh sách sơ bộ những người ứng cử ĐBQH và người ứng cử đại biểu Hội đồng nhân dân các cấp.Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia chủ trì phiên họp thứ 4 của Hội đồng.Như vậy, sau Hội nghị hiệp thương thứ hai, tổng số người ứng cử ĐBQH là 1.084 người, trong đó có 205 người do các cơ quan, tổ chức, đơn vị ở Trung ương giới thiệu, 803 người do các cơ quan, tổ chức, đơn vị ở địa phương giới thiệu, 76 người tự ứng cử, đạt tỉ lệ bình quân 2,17 lần so với tổng số ĐBQH được bầu...Khẩn trương triển khai các nhiệm vụ trong công tác bầu cửPhát biểu ý kiến kết luận phiên họp, Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia cho biết, Phiên họp thứ 4 Hội đồng bầu cử quốc gia đã hoàn thành các nội dung chương trình đề ra. Công tác chuẩn bị bầu cử đến nay trong phạm vi cả nước đã và đang được triển khai đúng pháp luật và bảo đảm theo tiến độ đề ra, chưa phát sinh vấn đề nào ảnh hưởng đến cuộc bầu cử.Năm đoàn kiểm tra đợt 1 của Hội đồng bầu cử quốc gia đánh giá các địa phương được kiểm tra đều rất trách nhiệm và thực hiện tốt những công việc theo đúng quy định hướng dẫn của Trung ương.Các cấp ủy đảng, chính quyền, Mặt trận Tổ quốc và các cơ quan, tổ chức, đoàn thể ở địa phương đã thực hiện rất tốt công tác chuẩn bị và đang khẩn trương nghiêm túc triển khai công việc chuẩn bị cho bầu cử.Tại Phiên họp thứ 4, Hội đồng bầu cử quốc gia đã thảo luận, tham gia góp ý những ý kiến về dự thảo báo cáo kết quả triển khai công tác bầu cử Quốc hội khóa XV và đại biểu HĐND các cấp nhiệm kỳ 2021-2026.Những ý kiến phát biểu tại Phiên họp thứ 4 tập trung vào các nội dung gồm: Xây dựng những kịch bản để phòng chống dịch và kiểm soát an toàn vệ sinh thực phẩm cũng như tổ chức bầu cử ở những nơi có cách ly y tế tập trung phòng, chống dịch Covid-19 thì cần tạo điều kiện thuận lợi, an toàn để cho người dân đi bỏ phiếu, kể cả những người đang được cách ly; về đối sách phòng chống các hoạt động xuyên tạc, phá hoại bầu cử; làm tốt công tác phòng chống cháy nổ trước và trong ngày bầu cử.Hội đồng bầu cử quốc gia thông qua nguyên tắc phân bổ đại biểu Trung ương về ứng cử ở địa phương cũng như rà soát, nhất trí với kết quả hiệp thương lần thứ hai. Chủ tịch Quốc hội đề nghị cơ quan chức năng tiếp tục chỉ đạo trong tình huống không chỉ có dịch bệnh, cần bổ sung khả năng về thiên tai có thể xảy ra vào thời điểm bầu cử trong tháng 5 tới. Các kịch bản phải chắc chắn, chặt chẽ.Theo Chủ tịch Quốc hội, việc chỉ đạo thành lập các tổ bầu cử, tập huấn bầu cử cũng như chỉ đạo giải quyết khiếu nại tố cáo đang được triển khai và sắp tới cần quan tâm hơn, nhất là khi đã xong Hội nghị hiệp thương lần thứ ba. Công tác này phải được thực hiện đúng quy định.Về việc bầu cử sớm, Chủ tịch Quốc hội đề nghị Văn phòng Hội đồng bầu cử quốc gia thông báo đến những địa phương có nhu cầu bầu cử sớm để đăng ký. Theo Luật Bầu cử ĐBQH và đại biểu Hội đồng nhân dân, việc bầu cử sớm do Hội đồng bầu cử quốc gia quyết định, khi bầu cử rồi mà bầu thiếu, muốn bầu cử lại hay không thì các địa phương sẽ báo cáo xin ý kiến Hội đồng bầu cử quốc gia và quyền cho bầu cử lại, bầu cử thêm cũng thuộc thẩm quyền của Hội đồng bầu cử quốc gia.Về quy định in ấn tài liệu, đấu thầu, Chủ tịch Quốc hội đề nghị Văn phòng Hội đồng bầu cử quốc gia có văn bản đề nghị Bộ Tài chính hướng dẫn bổ sung thêm trong trường hợp khẩn cấp cần in ấn không qua đấu thầu thì cũng cần có thêm có hướng dẫn.Chủ tịch Quốc hội cũng lưu ý một số công tác trong thực hiện về kê khai tài sản, hướng dẫn kê khai tài sản… cần được thực hiện theo đúng quy định của pháp luật.Về tuyên truyền, một số nơi có sáng kiến, nếu có sáng kiến tốt thì áp dụng rộng rãi cho cả nước. Chủ tịch Quốc hội đề nghị tăng cường tuyên truyền thêm, nhiều hơn, hiệu quả hơn cho đợt hội nghị hiệp thương lần thứ ba và cho tới ngày bầu cử 23/5 tới với mật độ tăng cường.Sau khi có kết quả hội nghị hiệp thương lần ba, sẽ lên danh sách chính thức những người ứng cử ĐBQH và Hội đồng nhân dân.Về bảo đảm cơ cấu ứng cử là nữ cho ĐBQH và Hội đồng nhân dân các cấp, hiện nay cố gắng thực hiện theo phương hướng bầu cử, tức là trong danh sách bầu cử có ít nhất 35% ứng cử viên là nữ. Chủ tịch Quốc hội cho rằng phải tiếp tục quan tâm tuyên truyền, nâng cao kiến thức, kỹ năng của các nữ ứng cử viên mà Trung ương Hội Liên hiệp Phụ nữ Việt Nam đã thực hiện rất tốt.Chủ tịch Quốc hội đề nghị Chánh Văn phòng Hội đồng bầu cử quốc gia tiếp thu đầy đủ các ý kiến của các thành viên Hội đồng bầu cử quốc gia để hoàn thiện các văn bản gửi tới các ĐBQH về báo cáo nhiệm kỳ, báo cáo của Hội đồng bầu cử quốc gia; đồng thời tập trung tham mưu giúp Hội đồng bầu cử quốc gia và các Tiểu ban đôn đốc theo dõi việc triển khai tiếp theo để chuẩn bị cho bầu cử.Các tiểu ban và Văn phòng của Hội đồng bầu cử quốc gia cũng như các cơ quan của Chính phủ, Ủy ban Trung ương Mặt trận Tổ quốc Việt Nam, các bộ, ban, ngành tiếp tục rà soát lại những công việc theo chức năng, nhiệm vụ được phân công để khẩn trương chỉ đạo, triển khai nhiệm vụ về công tác bầu cử...Tại phiên họp, Hội đồng bầu cử quốc gia đã xem xét thông qua hai nghị quyết đó là Nghị quyết về xử lý trường hợp khuyết người ứng cử Hội đồng nhân dân vì lý do bất khả kháng và Nghị quyết về nguyên tắc phân bổ ứng cử viên của Trung ương về địa phương ứng cử ĐBQH.Chủ tịch Quốc hội đề nghị Tiểu ban văn kiện, pháp luật và thông tin tuyên truyền, Tiểu ban nhân sự tiếp thu các ý kiến tại phiên họp để nhanh chóng hoàn thiện trình các văn bản. Riêng văn bản về nguyên tắc phân bổ đại biểu, đề nghị trình Chủ tịch Hội đồng bầu cử quốc gia sau khi được kiện toàn ký, để gắn với Hội nghị hiệp thương lần thứ ba...Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia nêu rõ, sau phiên họp, các cơ quan Chính phủ, Ủy ban Trung ương Mặt trận Tổ quốc Việt Nam, các bộ, ngành, địa phương và các cơ quan có liên quan tích cực chủ động trong công tác lãnh đạo, chỉ đạo. Việc nào cần hướng dẫn phải hướng dẫn sớm để cho các Ủy ban bầu cử biết, thực hiện và trả lời cho đúng.Theo TTXVN
'''

start_time = time.time()
similar_index_newspapers = get_similar_newspapers(text)
print(f'--- {time.time() - start_time} seconds ---')

print('Number of similar newspapers =', len(similar_index_newspapers))
for index_newspaper in similar_index_newspapers:
    print(newspapers[index_newspaper])

--- 1.15755009651184082 seconds ---
Number of similar newspapers = 1
1.136 hồ sơ ứng cử đại biểu Quốc hội.txt
